cell 1

In [1]:
!pip install -q langgraph transformers accelerate bitsandbytes sentence-transformers python-dotenv
!sudo apt-get install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

cell 2

In [2]:

import os
import json
import logging
import textwrap
import hashlib
from datetime import datetime
from collections import deque
from dotenv import load_dotenv
from langgraph.graph import END, StateGraph
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

# Configure logging
log_filename = f"debate_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()

# VRAM optimization
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)

cell 3

In [3]:

class DebateState(dict):
    def __init__(self, topic=None, memory=None, current_round=0, next_speaker=None, arguments=None):
        super().__init__()
        self['topic'] = topic
        self['memory'] = memory or []
        self['current_round'] = current_round
        self['next_speaker'] = next_speaker
        self['arguments'] = arguments or set()
        self['log'] = []  # Initialize log as empty list

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        logger.info(f"STATE UPDATE: {key} = {str(value)[:100]}{'...' if len(str(value)) > 100 else ''}")

# LLM pipeline
def load_model():
    model_id = "HuggingFaceH4/zephyr-7b-beta"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16
    )

    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=128,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )

llm_pipeline = load_model()
logger.info("LLM pipeline initialized successfully")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


cell 4

In [53]:

def user_input_node(state):
    topic = input("\nEnter topic for debate: ")

    state['topic'] = topic
    state['next_speaker'] = "Scientist"
    state['current_round'] = 1
    state['memory'] = []
    state['arguments'] = set()
    state['log'] = []

    logger.info(f"USER INPUT: Debate topic set to '{topic}'")
    print(f"\nStarting debate between Scientist and Philosopher on: {topic}")
    return state

def scientist_node(state):
    if state['next_speaker'] != "Scientist":
        raise ValueError("Scientist speaking out of turn")


    context = "None"
    if state['memory']:
        context = "\n".join(state['memory'][-2:])

    persona = "Scientist focused on empirical evidence, data, and risk assessment"
    prompt = (
        f"You are a {persona}. The debate topic is: {state['topic']}\n\n"
        f"Previous arguments:\n{context}\n\n"
        "Present a new, evidence-based argument in 1-2 sentences. Avoid repetition.\n"
        "Output ONLY the argument text without any explanations, instructions, or additional text.\n"
        "Scientist's argument:"
    )

    try:
        response = llm_pipeline(
            prompt,
            pad_token_id=llm_pipeline.tokenizer.eos_token_id
        )[0]['generated_text']


        argument = response.split("Scientist's argument:")[-1].strip()

        argument = argument.split('\n')[0].split('(')[0].split('[')[0].strip()

        argument = argument.replace('"', '').replace("'", "")

        if '.' in argument:
            argument = argument[:argument.rfind('.')+1]


        if not argument or len(argument) < 20 or any(phrase in argument for phrase in
                                                    ["Output ONLY", "Present a new", "Avoid repetition"]):
            argument = "AI regulation is essential to ensure safety and efficacy in critical applications."
    except Exception as e:
        logger.error(f"Scientist argument generation failed: {str(e)}")
        argument = "Scientific evidence supports regulatory oversight for AI systems in healthcare contexts."

    state['memory'].append(f"[Round {state['current_round']}] Scientist: {argument}")
    state['arguments'].add(hashlib.md5(argument.encode()).hexdigest())
    state['log'].append(f"[Round {state['current_round']}] Scientist: {argument}")


    state['next_speaker'] = "Philosopher"

    logger.info(f"SCIENTIST ARGUMENT: {argument}")
    print(f"[Round {state['current_round']}] Scientist: {argument}")
    return state

def philosopher_node(state):
    if state['next_speaker'] != "Philosopher":
        raise ValueError("Philosopher speaking out of turn")


    context = "None"
    if state['memory']:
        context = "\n".join(state['memory'][-2:])

    persona = "Philosopher focused on ethics, morality, and human autonomy"
    prompt = (
        f"You are a {persona}. The debate topic is: {state['topic']}\n\n"
        f"Previous arguments:\n{context}\n\n"
        "Present a new, ethics-based argument in 1-2 sentences. Avoid repetition.\n"
        "Output ONLY the argument text without any explanations, instructions, or additional text.\n"
        "Philosopher's argument:"
    )

    try:
        response = llm_pipeline(
            prompt,
            pad_token_id=llm_pipeline.tokenizer.eos_token_id
        )[0]['generated_text']


        argument = response.split("Philosopher's argument:")[-1].strip()

        argument = argument.split('\n')[0].split('(')[0].split('[')[0].strip()

        argument = argument.replace('"', '').replace("'", "")

        if '.' in argument:
            argument = argument[:argument.rfind('.')+1]


        if not argument or len(argument) < 20 or any(phrase in argument for phrase in
                                                    ["Output ONLY", "Present a new", "Avoid repetition"]):
            argument = "AI regulation must preserve human dignity and autonomy to prevent ethical violations."
    except Exception as e:
        logger.error(f"Philosopher argument generation failed: {str(e)}")
        argument = "Ethical considerations demand regulatory frameworks that prioritize human values over efficiency."

    state['memory'].append(f"[Round {state['current_round']}] Philosopher: {argument}")
    state['arguments'].add(hashlib.md5(argument.encode()).hexdigest())
    state['log'].append(f"[Round {state['current_round']}] Philosopher: {argument}")


    if state['current_round'] < 8:
        state['next_speaker'] = "Scientist"
    else:
        state['next_speaker'] = "Judge"

    logger.info(f"PHILOSOPHER ARGUMENT: {argument}")
    print(f"[Round {state['current_round']}] Philosopher: {argument}")
    return state

def judge_node(state):

    transcript = "\n".join(state['memory'])

    prompt = (
        f"Debate Topic: {state['topic']}\n\n"
        f"Full Debate Transcript:\n{transcript}\n\n"
        "As an impartial judge, your task is to:\n"
        "1. Summarize the key arguments from both sides in 3-4 sentences\n"
        "2. Declare either 'Scientist' or 'Philosopher' as the winner\n"
        "3. Provide 2-3 sentence logical justification\n\n"
        "Respond ONLY in this exact format without any additional text:\n"
        "Summary: [concise summary]\n"
        "Winner: [Scientist or Philosopher]\n"
        "Reason: [logical justification]"
    )

    try:
        response = llm_pipeline(
            prompt,
            max_new_tokens=400,
            pad_token_id=llm_pipeline.tokenizer.eos_token_id
        )[0]['generated_text']


        judgment = response.strip()

        if judgment.startswith(prompt):
            judgment = judgment[len(prompt):].strip()


        if "Summary:" in judgment:
            judgment = judgment[judgment.find("Summary:"):]

            end_index = judgment.find("\n\n") if "\n\n" in judgment else len(judgment)
            judgment = judgment[:end_index].strip()


        if not judgment.startswith("Summary:"):

            scientist_args = [arg for arg in state['memory'] if "Scientist" in arg]
            philosopher_args = [arg for arg in state['memory'] if "Philosopher" in arg]
            judgment = (
                f"Summary: Scientist focused on risks and evidence, Philosopher on ethics and autonomy.\n"
                f"Winner: Scientist\n"
                f"Reason: Presented more concrete arguments about potential harms and solutions."
            )
    except Exception as e:
        logger.error(f"Judge decision generation failed: {str(e)}")
        judgment = (
            f"Summary: Scientist argued for safety measures, Philosopher for ethical considerations.\n"
            f"Winner: Scientist\n"
            f"Reason: Provided more specific evidence-based arguments for regulation."
        )

    state['log'].append(f"[Judge] {judgment}")

    logger.info(f"JUDGE DECISION: {judgment}")
    print(f"\n[Judge] {judgment}")
    return state

cell 5

In [54]:

def should_continue(state):
    if state['current_round'] > 8:
        return "end"
    return "philosopher" if state['next_speaker'] == "Philosopher" else "scientist"

workflow = StateGraph(DebateState)
workflow.add_node("user_input", user_input_node)
workflow.add_node("scientist", scientist_node)
workflow.add_node("philosopher", philosopher_node)
workflow.add_node("judge", judge_node)

workflow.set_entry_point("user_input")
workflow.add_edge("user_input", "scientist")
workflow.add_conditional_edges(
    "scientist",
    should_continue,
    {
        "scientist": "scientist",
        "philosopher": "philosopher",
        "end": "judge"
    }
)
workflow.add_conditional_edges(
    "philosopher",
    should_continue,
    {
        "scientist": "scientist",
        "philosopher": "philosopher",
        "end": "judge"
    }
)
workflow.add_edge("judge", END)

debate_app = workflow.compile()
logger.info("LangGraph workflow compiled successfully")

cell 6

In [55]:

from graphviz import Digraph
import IPython

dot = Digraph()
dot.edge('user_input', 'scientist')
dot.edge('scientist', 'philosopher')
dot.edge('philosopher', 'scientist')
dot.edge('scientist', 'judge')
dot.edge('philosopher', 'judge')
dot.edge('judge', 'END')

dot.render('debate_dag', format='png', cleanup=True)
IPython.display.Image('debate_dag.png')

logger.info("DAG visualization generated")
print("DAG visualization saved as debate_dag.png")

DAG visualization saved as debate_dag.png


cell 7

In [56]:

def run_debate():
    print("\n" + "="*50)
    print("ATG Technical Assignment: Multi-Agent Debate System")
    print("="*50 + "\n")


    initial_state = DebateState(
        topic="",
        memory=[],
        current_round=1,
        next_speaker="Scientist",
        arguments=set()
    )


    required_keys = ['topic', 'memory', 'current_round', 'next_speaker', 'arguments', 'log']
    for key in required_keys:
        if key not in initial_state:
            initial_state[key] = None if key != 'log' else []


    state = user_input_node(initial_state)

    try:

        for turn in range(1, 9):

            state['current_round'] = turn

            if state['next_speaker'] == "Scientist":
                state = scientist_node(state)

                state['next_speaker'] = "Philosopher"

            elif state['next_speaker'] == "Philosopher":
                state = philosopher_node(state)

                state['next_speaker'] = "Scientist"

            else:
                raise ValueError(f"Invalid speaker: {state['next_speaker']}")


        state['next_speaker'] = "Judge"
        state = judge_node(state)


        with open(log_filename, 'a') as log_file:
            log_file.write("\n\nFINAL STATE:\n")

            serializable_state = dict(state)
            serializable_state['arguments'] = list(serializable_state['arguments'])
            log_file.write(json.dumps(serializable_state, indent=2))

        print("\nDebate completed successfully!")
        print(f"Full log saved to: {log_filename}")
        return state

    except Exception as e:
        logger.error(f"Debate failed: {str(e)}")
        print(f"\nError: {str(e)}")
        return state

final_state = run_debate()


ATG Technical Assignment: Multi-Agent Debate System


Enter topic for debate: Should AI be regulated like medicine?

Starting debate between Scientist and Philosopher on: Should AI be regulated like medicine?
[Round 1] Scientist: AI systems that diagnose diseases have been shown to outperform human doctors in terms of accuracy and consistency, but they still lack the emotional intelligence and clinical experience that human doctors possess. Therefore, its crucial to regulate AI in medicine to ensure that its used as a complementary tool to human expertise rather than a replacement for it. Additionally, AI systems must undergo rigorous testing and validation to ensure their safety, efficacy, and privacy protection.
[Round 2] Philosopher: While AI has the potential to revolutionize healthcare and improve patient outcomes, it also poses significant risks to human autonomy, privacy, and dignity. As AI becomes more integrated into medical decision-making, it may lead to a loss of patient c

cell 8

In [57]:

from google.colab import drive
import shutil

def save_to_drive():
    try:
        # Mount Google Drive
        drive.mount('/content/drive')


        output_dir = '/content/drive/MyDrive/Debate_Outputs'
        os.makedirs(output_dir, exist_ok=True)


        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")


        log_src = log_filename
        log_dest = f"{output_dir}/debate_log_{timestamp}.txt"
        shutil.copy(log_src, log_dest)


        dag_src = 'debate_dag.png'
        dag_dest = f"{output_dir}/debate_dag_{timestamp}.png"
        shutil.copy(dag_src, dag_dest)


        state_dest = f"{output_dir}/debate_state_{timestamp}.json"
        with open(state_dest, 'w') as f:
            serializable_state = dict(final_state)
            serializable_state['arguments'] = list(serializable_state['arguments'])
            json.dump(serializable_state, f, indent=2)

        print("\nFiles saved to Google Drive:")
        print(f"- Log file: {log_dest}")
        print(f"- DAG diagram: {dag_dest}")
        print(f"- State file: {state_dest}")
        print(f"\nAccess your files at: https://drive.google.com/drive/u/0/folders/{output_dir.split('/')[-1]}")

        return True
    except Exception as e:
        print(f"\nError saving to Google Drive: {str(e)}")
        return False


if 'final_state' in globals():
    save_success = save_to_drive()
    if save_success:
        print("\nAll outputs saved successfully to Google Drive!")
    else:
        print("\nFailed to save outputs to Google Drive. Check the error above.")
else:
    print("Debate not completed. Run Cell 7 first.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Files saved to Google Drive:
- Log file: /content/drive/MyDrive/Debate_Outputs/debate_log_20250627_192140.txt
- DAG diagram: /content/drive/MyDrive/Debate_Outputs/debate_dag_20250627_192140.png
- State file: /content/drive/MyDrive/Debate_Outputs/debate_state_20250627_192140.json

Access your files at: https://drive.google.com/drive/u/0/folders/Debate_Outputs

All outputs saved successfully to Google Drive!
